In [ ]:
# 1. Kütüphaneleri Kur
!pip install -q -U google-generativeai PyPDF2 gradio Pillow

import google.generativeai as genai
import PyPDF2
import gradio as gr
from google.colab import userdata
from PIL import Image
import os

# 2. API VE AKILLI MODEL SEÇİMİ
try:
    api_key = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=api_key)

    # Erişilebilir modelleri bulunması ve en yenisinin seçilmesi
    modeller = [m.name for m in genai.list_models() if 'generateContent' in m.supported_generation_methods]

    # Öncelik sırası: 1.5-flash (hızlı/görsel), değilse 1.5-pro, o da yoksa ilk bulunan
    if 'models/gemini-1.5-flash' in modeller:
        secilen_model = 'models/gemini-1.5-flash'
    elif 'models/gemini-pro' in modeller:
        secilen_model = 'models/gemini-pro'
    else:
        secilen_model = modeller[0]

    model = genai.GenerativeModel(secilen_model)
    print(f"✅ Bağlantı Başarılı! Kullanılan Model: {secilen_model}")
except Exception as e:
    print(f"❌ API Hatası: {e}")

# 3. VERİ SAKLAMA
hafiza = {"veri": None, "tur": None}

def dosya_isle(file):
    global hafiza
    if file is None: return "⚠️ Dosya bekleniyor..."

    uzanti = os.path.splitext(file.name)[1].lower()
    try:
        if uzanti == ".pdf":
            reader = PyPDF2.PdfReader(file.name)
            metin = " ".join([p.extract_text() or "" for p in reader.pages])
            hafiza["veri"] = metin[:20000]
            hafiza["tur"] = "pdf"
            return "✅ PDF Analiz Edildi. Sorunları yanıtlamaya hazırım."
        else:
            hafiza["veri"] = Image.open(file.name)
            hafiza["tur"] = "gorsel"
            return "✅ Görsel/Fotoğraf Algılandı. Belgeyi tartışabiliriz."
    except Exception as e:
        return f"❌ Okuma hatası: {e}"

def sohbet(mesaj, history):
    global hafiza
    if not hafiza["tur"]: return "Lütfen önce bir belge yükleyin."

    try:
        if hafiza["tur"] == "gorsel":
            # Görsel için multimodal istek
            res = model.generate_content([f"Hukuk asistanı olarak yanıtla: {mesaj}", hafiza["veri"]])
        else:
            # PDF metni için istek
            res = model.generate_content(f"Belge içeriği: {hafiza['veri']}\n\nSoru: {mesaj}")

        return res.text
    except Exception as e:
        return f"⚠️ API Yanıt Vermedi: {e}\n\nNot: API anahtarınızın günlük kotası dolmuş olabilir."

# 4. ARAYÜZ
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# ⚖️ AI Avukatlık Şirketi Asistanı")
    with gr.Row():
        yukle = gr.File(label="Dosya veya Görsel Atın")
        bilgi = gr.Textbox(label="Sistem Mesajı", interactive=False)

    yukle.change(dosya_isle, inputs=yukle, outputs=bilgi)
    gr.ChatInterface(fn=sohbet, title="Hukuki Analiz Paneli")

demo.launch(debug=True)

✅ Bağlantı Başarılı! Kullanılan Model: models/gemini-2.5-flash


/tmp/ipython-input-804756904.py:71: UserWarning: The parameters have been moved from the Blocks constructor to the launch() method in Gradio 6.0: theme. Please pass these parameters to launch() instead.
  with gr.Blocks(theme=gr.themes.Soft()) as demo:


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://64a61d25db5067e476.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
